In [1]:
import re
from io import StringIO
from datetime import datetime, timedelta

import requests
import pandas as pd


class YahooFinanceHistory:
    timeout = 2
    crumb_link = 'https://finance.yahoo.com/quote/{0}/history?p={0}'
    crumble_regex = r'CrumbStore":{"crumb":"(.*?)"}'
    quote_link = 'https://query1.finance.yahoo.com/v7/finance/download/{quote}?period1={dfrom}&period2={dto}&interval=1d&events=history&crumb={crumb}'

    def __init__(self, symbol, days_back=7):
        self.symbol = symbol
        self.session = requests.Session()
        self.dt = timedelta(days=days_back)

    def get_crumb(self):
        response = self.session.get(self.crumb_link.format(self.symbol), timeout=self.timeout)
        response.raise_for_status()
        match = re.search(self.crumble_regex, response.text)
        if not match:
            raise ValueError('Could not get crumb from Yahoo Finance')
        else:
            self.crumb = match.group(1)

    def get_quote(self):
        if not hasattr(self, 'crumb') or len(self.session.cookies) == 0:
            self.get_crumb()
        now = datetime.utcnow()
        dateto = int(now.timestamp())
        datefrom = int((now - self.dt).timestamp())
        url = self.quote_link.format(quote=self.symbol, dfrom=datefrom, dto=dateto, crumb=self.crumb)
        response = self.session.get(url)
        response.raise_for_status()
        return pd.read_csv(StringIO(response.text), parse_dates=['Date'])

In [5]:
stocks = pd.read_csv('SP500index.csv')
df = YahooFinanceHistory('A', days_back=3500).get_quote()
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-09-11,19.942776,20.143061,19.849785,20.064377,18.480059,6138300
1,2009-09-14,19.928469,20.157368,19.771101,20.100143,18.512999,4913100
2,2009-09-15,19.778255,20.300428,19.778255,20.257511,18.657946,4638700
3,2009-09-16,20.264664,20.579399,20.264664,20.479256,18.862179,4520000
4,2009-09-17,20.450644,20.615164,20.157368,20.379112,18.769939,5478700
5,2009-09-18,20.522175,20.550787,19.899857,20.293276,18.690886,6202500
6,2009-09-21,20.550787,21.015738,20.472103,20.608011,18.980772,7854900
7,2009-09-22,20.715307,20.715307,20.307583,20.407724,18.796299,6613600
8,2009-09-23,20.450644,20.622318,20.135908,20.221745,18.624998,6227800
9,2009-09-24,20.300428,20.350500,19.728184,19.907009,18.335117,4159100


In [10]:
#stocks['Symbol']
tickers = stocks['Symbol'].tolist()
#df
tickers
for ticker in tickers:
    print (ticker)
    for i in range(10):
        try:
            #print("...")
            df = YahooFinanceHistory(ticker, days_back=3500).get_quote()
            filename = "SP500Stocks/"+ticker+".csv"
            print(filename)
            df.to_csv(filename, index=False)
            break
        except:
            #print("?")
            continue

A
SP500Stocks/A.csv
AAL
SP500Stocks/AAL.csv
AAP
SP500Stocks/AAP.csv
AAPL
SP500Stocks/AAPL.csv
ABBV
SP500Stocks/ABBV.csv
ABC
SP500Stocks/ABC.csv
ABMD
SP500Stocks/ABMD.csv
ABT
SP500Stocks/ABT.csv
ACN
SP500Stocks/ACN.csv
ADBE
SP500Stocks/ADBE.csv
ADI
SP500Stocks/ADI.csv
ADM
SP500Stocks/ADM.csv
ADP
SP500Stocks/ADP.csv
ADS
SP500Stocks/ADS.csv
ADSK
SP500Stocks/ADSK.csv
AEE
SP500Stocks/AEE.csv
AEP
SP500Stocks/AEP.csv
AES
SP500Stocks/AES.csv
AFL
SP500Stocks/AFL.csv
AGN
SP500Stocks/AGN.csv
AIG
SP500Stocks/AIG.csv
AIV
SP500Stocks/AIV.csv
AIZ
SP500Stocks/AIZ.csv
AJG
SP500Stocks/AJG.csv
AKAM
SP500Stocks/AKAM.csv
ALB
SP500Stocks/ALB.csv
ALGN
SP500Stocks/ALGN.csv
ALK
SP500Stocks/ALK.csv
ALL
SP500Stocks/ALL.csv
ALLE
SP500Stocks/ALLE.csv
ALXN
SP500Stocks/ALXN.csv
AMAT
SP500Stocks/AMAT.csv
AMD
SP500Stocks/AMD.csv
AME
SP500Stocks/AME.csv
AMG
SP500Stocks/AMG.csv
AMGN
SP500Stocks/AMGN.csv
AMP
SP500Stocks/AMP.csv
AMT
SP500Stocks/AMT.csv
AMZN
SP500Stocks/AMZN.csv
ANET
SP500Stocks/ANET.csv
ANSS
SP500Stocks/A

SP500Stocks/NKE.csv
NKTR
SP500Stocks/NKTR.csv
NLSN
SP500Stocks/NLSN.csv
NOC
SP500Stocks/NOC.csv
NOV
SP500Stocks/NOV.csv
NRG
SP500Stocks/NRG.csv
NSC
SP500Stocks/NSC.csv
NTAP
SP500Stocks/NTAP.csv
NTRS
SP500Stocks/NTRS.csv
NUE
SP500Stocks/NUE.csv
NVDA
SP500Stocks/NVDA.csv
NWL
SP500Stocks/NWL.csv
NWS
SP500Stocks/NWS.csv
NWSA
SP500Stocks/NWSA.csv
O
SP500Stocks/O.csv
OKE
SP500Stocks/OKE.csv
OMC
SP500Stocks/OMC.csv
ORCL
SP500Stocks/ORCL.csv
ORLY
SP500Stocks/ORLY.csv
OXY
SP500Stocks/OXY.csv
PAYX
SP500Stocks/PAYX.csv
PBCT
SP500Stocks/PBCT.csv
PCAR
SP500Stocks/PCAR.csv
PEG
SP500Stocks/PEG.csv
PEP
SP500Stocks/PEP.csv
PFE
SP500Stocks/PFE.csv
PFG
SP500Stocks/PFG.csv
PG
SP500Stocks/PG.csv
PGR
SP500Stocks/PGR.csv
PH
SP500Stocks/PH.csv
PHM
SP500Stocks/PHM.csv
PKG
SP500Stocks/PKG.csv
PKI
SP500Stocks/PKI.csv
PLD
SP500Stocks/PLD.csv
PM
SP500Stocks/PM.csv
PNC
SP500Stocks/PNC.csv
PNR
SP500Stocks/PNR.csv
PNW
SP500Stocks/PNW.csv
PPG
SP500Stocks/PPG.csv
PPL
SP500Stocks/PPL.csv
PRGO
SP500Stocks/PRGO.csv
PRU
SP